In [ ]:
# #한글 폰트를 설치합니다.
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
# URL에 있는 파일을 다운로드 받습니다.

import urllib.request

urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/tykimos/tykimos.github.io/master/warehouse/dataset/tarr_train.txt",
    filename="tarr_train.txt",
)

In [ ]:
# 정형데이터 분석 >> 판다스

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#load file
train_file_path='/content/tarr_train.txt'

In [ ]:
pd.read_csv(train_file_path,sep='\t')

In [ ]:
train_data=pd.read_csv(train_file_path,sep='\t')

train_data.head(3)

In [ ]:
len(train_data)

In [ ]:
train_data['comment'].apply(len).head(5)

In [ ]:
#댓글의 길이가 길면 거의 긍정이 아닐까?
train_data['comment_length']=train_data['comment'].apply(len)

In [ ]:
train_data.head(3)

In [ ]:
train_data['comment_length'].describe()

In [ ]:
comment_length_statistics=train_data['comment_length'].describe()
comment_length_statistics

In [ ]:
# matplotlib : 데이터를 시각화하는데 많이 사용되는 패키지입니다.

import matplotlib.pyplot as plt

# Plotting the distribution of comment lengths
plt.figure(figsize=(12, 6))
plt.hist(train_data['comment_length'], bins=30, color='blue', alpha=0.7)
plt.axvline(train_data['comment_length'].mean(), color='red', linestyle='dashed', linewidth=1, label='Average Length')
plt.title('Distribution of Comment Lengths')
plt.xlabel('Comment Length')
plt.ylabel('Number of Comments')
plt.legend()
plt.grid(axis='y')
plt.show()

In [ ]:
train_data['label'].value_counts()

In [ ]:
label_count=train_data['label'].value_counts()
print(label_count)

In [ ]:
label_distribution=train_data['label'].value_counts(normalize=True)
print(label_distribution)

In [ ]:
# Plotting distribution of comment lengths by label
plt.figure(figsize=(12, 6))
plt.hist(train_data[train_data['label'] == 1]['comment_length'], bins=30, color='blue', alpha=0.7, label='Positive Comments')
plt.hist(train_data[train_data['label'] == 0]['comment_length'], bins=30, color='red', alpha=0.5, label='Negative Comments')
plt.title('Distribution of Comment Lengths by Label')
plt.xlabel('Comment Length')
plt.ylabel('Number of Comments')
plt.legend()
plt.grid(axis='y')
plt.show()

In [ ]:
from collections import Counter
import re

def get_word_frequencies(texts):
    words=[]
    for text in texts:
        #맞춤점 제거한 후 공백기준으로 토큰화
        words.extend(re.findall(r'\b\w+\b',text))
    #단어 빈도수 계산
    return Counter(words)

In [ ]:
train_data[train_data['label']==1]['comment'].head(10)

In [ ]:
positive_comments=train_data[train_data['label']==1]['comment']
negative_comments=train_data[train_data['label']==0]['comment']
negative_comments[:3]

In [ ]:
print(get_word_frequencies(positive_comments))

In [ ]:
positive_word_freq=get_word_frequencies(positive_comments)
negative_word_freq=get_word_frequencies(negative_comments)

In [ ]:
print(negative_word_freq)

In [ ]:
positive_word_freq.most_common(10)

In [ ]:
negative_word_freq.most_common(10)

In [ ]:
top_positive_words=positive_word_freq.most_common(10)
top_negative_words=negative_word_freq.most_common(10)

In [ ]:
from wordcloud import WordCloud

# Generate word clouds for positive and negative comments
def generate_wordcloud(word_freq, title):
    wc = WordCloud(width=400, height=400, background_color='white', font_path='NanumGothic.ttf').generate_from_frequencies(word_freq)
    plt.figure(figsize=(6, 6))
    plt.imshow(wc, interpolation='bilinear')
    plt.title(title)
    plt.axis('off')
    plt.show()

# Displaying word clouds
generate_wordcloud(positive_word_freq, 'Word Cloud for Positive Comments')
generate_wordcloud(negative_word_freq, 'Word Cloud for Negative Comments')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

#텍스트 모음에서 상위 n개의 n-gram 반환함수
def get_top_ngrams(corpus,ngram_range,top_n):
    #초기화
    vec=CountVectorizer(ngram_range=ngram_range).fit(corpus)
    bag_of_words= vec.transform(corpus)
    #결과물: ngram의 빈도를 나타내는 행렬
    sum_words=bag_of_words.sum(axis=0)
    words_freq=[(word,sum_words[0,idx]) for word,idx in vec.vocabulary_.items()]
    words_freq=sorted(words_freq,key=lambda x:x[1],reverse=True) # 내림차순 정렬
    #x =(word, freq) >> tuple 2번째인 빈도수 기준
    return words_freq[:top_n]

In [ ]:
#top bigram, top trigram
top_digram=get_top_ngrams(train_data['comment'],ngram_range=(2,2),top_n=10)
top_triigram=get_top_ngrams(train_data['comment'],ngram_range=(3,3),top_n=10)

In [ ]:
top_digram

In [ ]:
top_triigram

In [ ]:
get_top_ngrams(train_data[train_data['label']==1]['comment'],ngram_range=(2,2),top_n=10)

In [ ]:
top_bigrams_positive=get_top_ngrams(train_data[train_data['label']==1]['comment'],ngram_range=(2,2),top_n=10)
top_trigrams_positive=get_top_ngrams(train_data[train_data['label']==1]['comment'],ngram_range=(3,3),top_n=10)

top_bigrams_negative=get_top_ngrams(train_data[train_data['label']==0]['comment'],ngram_range=(2,2),top_n=10)
top_trigrams_negative=get_top_ngrams(train_data[train_data['label']==0]['comment'],ngram_range=(3,3),top_n=10)

In [ ]:
top_bigrams_positive,top_trigrams_positive

In [ ]:
top_bigrams_negative,top_trigrams_negative

In [ ]:
keywords=['서비스','가격','맛','분위기','직원']

keyword_counts={keyword:train_data['comment'].str.count(keyword).sum() for keyword in keywords}
keyword_counts

In [ ]:
{keyword:train_data[train_data['label']==1]['comment'].str.count(keyword).sum() for keyword in keywords}

In [ ]:
positive_keyword_counts={keyword:train_data[train_data['label']==1]['comment'].str.count(keyword).sum() for keyword in keywords}
negative_keyword_counts={keyword:train_data[train_data['label']==0]['comment'].str.count(keyword).sum() for keyword in keywords}

In [ ]:
positive_keyword_counts,negative_keyword_counts

In [ ]:
!pip install koreanize-matplotlib

In [ ]:
import koreanize_matplotlib

# 그래프에 retina display 적용
%config InlineBackend.figure_format = 'retina'

pd.Series([1, 3, 5, -7, 9]).plot(title="한글", figsize=(6, 1))

In [ ]:
import seaborn as sns

# Visualizing keyword frequency for positive and negative labels

# Plotting
fig, ax = plt.subplots(2, 1, figsize=(8,8))

sns.set(font="NanumGothic", rc={"axes.unicode_minus":False}, style='white')

# Positive reviews
sns.barplot(x=list(positive_keyword_counts.keys()), y=list(positive_keyword_counts.values()), ax=ax[0], palette='Blues_d')
ax[0].set_title("Keyword Frequency in Positive Reviews")
ax[0].set_ylabel("Frequency")

# Negative reviews
sns.barplot(x=list(negative_keyword_counts.keys()), y=list(negative_keyword_counts.values()), ax=ax[1], palette='Reds_d')
ax[1].set_title("Keyword Frequency in Negative Reviews")
ax[1].set_ylabel("Frequency")

plt.tight_layout()
plt.show()

In [ ]:
#tf-idf로 가중치를 줘야함

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [ ]:
#텍스트 코퍼스에서 점수가 가장 높은 상위 top_n개의 단어를 반환하는 함수
def get_top_tfidf_features(corpus,top_n):
    #tfidf 행렬 생성
    tfidf_vec=TfidfVectorizer().fit(corpus)
    transformed=tfidf_vec.transform(corpus)
    mean_tfidf=transformed.mean(axis=0).tolist()[0]

    #transformed 행렬의 각 열의 평균 tfidf 값 계산>>리스트로 변환
    sorted_tfidf_index=np.argsort(mean_tfidf)[-top_n:] #상위 top_n

    #상위 top_n의 단어에 해당하는 평균 tfidf 값을 리스트로 담기
    sorted_tfidf_values=[mean_tfidf[i] for i in sorted_tfidf_index]

    #상위 top_n개의 단어 차제를 담은 리스트
    sorted_tfidf_features=[tfidf_vec.get_feature_names_out()[i] for i in sorted_tfidf_index]

    #상위 top_n개의 단어와 그에 대응하는 tfidf 값>> 튜플 형태 결합
    return list(zip(sorted_tfidf_features, sorted_tfidf_values))


In [ ]:
get_top_tfidf_features(train_data['comment'],10)

In [ ]:
#tfidf 평균 상위10개 뽑아서 tfidf 값 리스트에 담고 그에 해당하는 단어 리스트에 담기
top_tfidf_features=get_top_tfidf_features(train_data['comment'],10)
top_tfidf_features

In [ ]:
get_top_tfidf_features(train_data[train_data['label']==1]['comment'],10)

In [ ]:
top_tfidf_positive=get_top_tfidf_features(train_data[train_data['label']==1]['comment'],10)
top_tfidf_negative=get_top_tfidf_features(train_data[train_data['label']==0]['comment'],10)

In [ ]:
top_tfidf_positive

In [ ]:
top_tfidf_negative

In [ ]:
corpus = [
    "I love machine learning and artificial intelligence",
    "Machine learning is fascinating",
    "I love learning new things about Gen-AI and ML"
]

In [ ]:
get_top_tfidf_features(corpus,5)

In [ ]:
#argsort
import numpy as np

arr=np.array([10,3,5,2])

np.argsort(arr) #원래 위치의 인덱스를 찾음

##추가학습

In [ ]:
#argsort: 기존 리스트에서의 인덱스(위치정보) 불러옴
import numpy as np

arr=np.array([10,3,5,2])
#[2,3,5,10]
np.argsort(arr)

In [ ]:
arr.sort()

In [ ]:
arr